# importing the required libraries

In [ ]:
from pathlib import Path
from collections import namedtuple
import numpy as np

from svspyed.utils.helper_functions import (
    check_csv_columns, get_layering_df, populate_layering_df,
)
from svspyed.input_preparation.prep_svs import ModelInputData
from svspyed.model.svs_model import SVSModel
from svspyed.ensemble_run.perturb_run import PerturbAndRun


# Variable definitions


In [ ]:
# size of the ensemble
ENS_SIZE = 10

# (max) number of processors to use for parallelization
N_PROCS = 4

# random seed
RND_SEED = 1915

# relevant paths
paths = dict(
    # input meteorological data required to drive SVS
    hourly_meteo = Path(
        "/Users/pinovanrijn/Desktop/repos/svspyed/svspyed/tests/test_case_study/data/"
        "hourly_meteo.csv"
    ),

    # SVS executable (compiled from Fortran source code)
    svs_exec = Path(
        "/Users/pinovanrijn/Desktop/repos/MESH_SVS/svs_jun12_2024"
    ),

    # the working dir where the run-time files will be stored
    working_dir = Path(
        "/Users/pinovanrijn/Desktop/no_sync/svs_test_run"
    ),

    # .met ensemble dir
    met_ensemble_dir = Path(
        "/Users/pinovanrijn/Desktop/Work/Manuscripts/Efforts/DeepPercolationEvaluationSVS/Scripts/GenMeteoEns/PerturbedForcingData"
    ),

)
# --------------------------------------------------------------------------- #

# a container for the properties of a soil type
SoilType = namedtuple("SoilType", [
    'code',
    'sand', 'clay',
    'wsat', 'wfc', 'wwilt',
    'ksat',
    'psisat', 'bcoef',
    'rhosoil',
])

# a container for a soil cover
Enclosure = namedtuple("Enclosure", [
    'layering',
    'soil_layers'
])

# a container for site-specific parameters required to configure SVS
SiteParams = namedtuple("SiteParams", [
    'deglat', 'deglng',
    'slop', 'zusl', 'ztsl',
    'observed_forcing', 'draindens', 'vf_type'
])

# a container for SVS internal parameters
SVSParams = namedtuple("SVSParams", [
    'SCHMSOL', 'lsoil_freezing_svs1', 'soiltext', 'read_user_parameters',
    'save_minutes_csv', 'water_ponding', 'KFICE', 'OPT_SNOW', 'OPT_FRAC',
    'OPT_LIQWAT', 'WAT_REDIS', 'tperm', 'user_wfcdp',
    "OPT_VEGFLUX", "OPT_AGGFLUX", "OPT_VEGCOND",
    "user_VEGDAT_INDEX", "user_VEGDAT_VALUE",
    "user_LAM_VEGL_STAB", "user_LAM_VEGL_UNSTAB",

])
# --------------------------------------------------------------------------- #

# checks #
# check if the paths lead to existing files/directories
for key, path in paths.items():
    if not path.exists():
        raise FileNotFoundError(f"{key} path does not exist: {path}")

# Setting up case study input object


In [ ]:
# Cover Material soil
CMsoil = SoilType(
    code = "CM",
    sand = 66.2, clay = 7.7, # percent
    wsat = 0.329, wfc = 0.05, wwilt = 0.01, # volumetric fraction
    ksat = 9.49E-06, # m.s-1
    psisat = 0.34, bcoef = 1.48, # mH2O, unitless
    rhosoil=1604.91, # kg.m-3
)


E1 = Enclosure(
    layering    = "190:5:CM",
    soil_layers = [CMsoil],

    # "190:5:CM" -> 190 cm total depth, divided into 5 cm increments, each with
    # the properties of CMsoil
    # could be changed to, for example, `15:2.5:CM + 175:5:CM` to have a 15 cm
    # top layer with 2.5 cm increments, and a 175 cm bottom layer with 5 cm
)

# parameters describing the site, identical for all enclosures
site_params = SiteParams(
    deglat = 45.82, deglng = -72.37, # degrees latitude and longitude

    slop = 0.02, zusl = 10.0, ztsl = 1.5, # slope, heights for momentum and temp
    observed_forcing = "height", draindens = 0.0, vf_type = 13
    # drainage density, vegetation fraction type
)

# check SVS source code and doc for more info on these parameters
svs_params = SVSParams(
    SCHMSOL = "SVS", lsoil_freezing_svs1 = ".TRUE.", soiltext = "NIL",
    read_user_parameters = 1, save_minutes_csv = 0, water_ponding = 0,
    KFICE = 0, OPT_SNOW = 2, OPT_FRAC = 1, OPT_LIQWAT = 1, WAT_REDIS = 0,
    tperm = 280.15, user_wfcdp = CMsoil.wsat * 0.99,
    OPT_VEGFLUX = 3, OPT_AGGFLUX = 2, OPT_VEGCOND = 2,
    user_VEGDAT_INDEX = 13, user_VEGDAT_VALUE = 0.7,
    user_LAM_VEGL_STAB = 10.0, user_LAM_VEGL_UNSTAB = 10.0,
)

# name of the directory containing the enclosure
HOST_DIR_NAME = "test_run"

# mapping of the required meteo variables to the labels in the forcing file
# needed for the `save_csv_as_met` function
meteo_cols = {
    "utc_dtime": "datetime_utc",
    "air_temperature": "Air temperature (degC)",
    "precipitation": "Precipitation (mm)",
    "wind_speed": "Wind speed (m.s-1)",
    "atmospheric_pressure": "Atmospheric pressure (Pa)",
    "shortwave_radiation": "Shortwave radiation (W.m-2)",
    "longwave_radiation": "Longwave radiation (W.m-2)",
    "specific_humidity": "Specific humidity (kg.kg-1)",
    "relative_humidity": "Relative humidity (%)"
}


# simulation start_date
START_DATE = "2018-182-04-00"
END_DATE = "2020-182-04-00"

# spinup end date
SPINUP_END_DATE = "2019-07-01 00:00:00"

# spinup date timezone
SPINUP_END_DATE_TZ = "America/Montreal"

# name of the parameters to assign for each layer: all fields except `code`
layer_params = tuple(field for field in CMsoil._fields if field != 'code')

# --------------------------------------------------------------------------- #
# processing the information to create the required files # ----------------- #

# read layering information and populate the dataframe
dfenc = get_layering_df(E1)
dfenc = populate_layering_df(dfenc, E1, site_params)

input_object_deterministic = ModelInputData(
    work_dir_path=paths['working_dir'],
    soilcover_info=dfenc,
    metfile_path=paths['hourly_meteo'],
    exec_file_path=paths['svs_exec'],
    host_dir_name=HOST_DIR_NAME,
    meteo_col_names=meteo_cols,
    param_col_names={
        **{k: k for k in layer_params},
        **{k: k for k in site_params._fields},
    },
    model_params=svs_params._asdict(),
    start_date=START_DATE,
    end_date=END_DATE,
    spinup_end_date=SPINUP_END_DATE,
    time_zone=SPINUP_END_DATE_TZ,
)

test_svs = SVSModel(input_object_deterministic, True, True)

# Checks # ------------------------------------------------------------------ #
# check if the columns in the meteo file are correct
print("Checking meteo file columns...")
check_csv_columns(paths['hourly_meteo'], meteo_cols)

# Creating ensemble scenarios


In [ ]:
# random number generator
drng = np.random.default_rng(RND_SEED)

# we only perturb `sand` and `clay` parameters
sand_range = (50, 70)
clay_range = (5, 10)

# number of soil layers defined in the layering information
n_layers = input_object_deterministic.soilcover_info.shape[0]


scenarios = {}
for ith_scn in range(3):

    # assumes soil column is homogeneous
    scenarios[f"parameter_scenario_{ith_scn}"] = {
        'sand': np.full(n_layers, drng.uniform(*sand_range)).round(2),
        'clay': np.full(n_layers, drng.uniform(*clay_range)).round(2),
    }

met_files = list(paths['met_ensemble_dir'].glob("*.met"))

print(
    F"Number of met files: {len(met_files)}\n"
    F"Number of scenarios: {len(scenarios)}"
)

# run the ensemble simulation


In [ ]:
p1 = PerturbAndRun(
    svs_default_input=input_object_deterministic,
    parameter_scenarios=scenarios,
    met_paths=met_files,
    njobs=6,
    verbose=True,
)

p1.run_all_parallel()